In [4]:
import json

file_path = 'result/CQ_Solver_summary_2.json'

# 儲存加總的總 token 數
total_prompt_tokens = 0
total_completion_tokens = 0
total_total_tokens = 0

# 計數每個 obj（也就是每個 agent 運行）
obj_count = 0

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict) and "token_usage" in item and isinstance(item["token_usage"], list):
                obj_prompt = 0
                obj_completion = 0
                obj_total = 0

                for usage in item["token_usage"]:
                    obj_prompt += usage.get("prompt_tokens", 0)
                    obj_completion += usage.get("completion_tokens", 0)
                    obj_total += usage.get("total_tokens", 0)

                total_prompt_tokens += obj_prompt
                total_completion_tokens += obj_completion
                total_total_tokens += obj_total
                obj_count += 1

        # 印出總計與平均值
        print("總 token 數量：")
        print(f"  Prompt Tokens: {total_prompt_tokens}")
        print(f"  Completion Tokens: {total_completion_tokens}")
        print(f"  Total Tokens: {total_total_tokens}")

        if obj_count > 0:
            print("\n平均每個物件（agent 運行）的 token 數量：")
            print(f"  Prompt Tokens: {total_prompt_tokens / obj_count:.2f}")
            print(f"  Completion Tokens: {total_completion_tokens / obj_count:.2f}")
            print(f"  Total Tokens: {total_total_tokens / obj_count:.2f}")
        else:
            print("沒有有效的 agent 運行資料。")
    else:
        print(f"錯誤：JSON 文件 '{file_path}' 的頂層結構不是列表，而是 {type(data)}")

except Exception as e:
    print(f"讀取或處理 JSON 文件時出錯：{e}")


總 token 數量：
  Prompt Tokens: 5861096
  Completion Tokens: 248502
  Total Tokens: 6109598

平均每個物件（agent 運行）的 token 數量：
  Prompt Tokens: 58610.96
  Completion Tokens: 2485.02
  Total Tokens: 61095.98


In [6]:
import json
with open("CQ_Solver_summary_2.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

for message in data[12]["conversations"]:
    if "agent" in message and message["agent"] == "planner" and message["role"] != 'developer':
        if "content" in message:
            print(message["content"])
        else:
            print(message["tool_calls"][0]["function"]["arguments"])
        print("------------------------------------------------------")

{
  "nodes": [
    {
      "node_id": "Q",
      "question": "how childhood shapes identity",
      "annotation": ""
    }
  ],
  "edges": []
}
------------------------------------------------------
{"graph":[{"parent_question_id":"Q","sub_question":"What role does family environment play in shaping childhood identity?","edge_annotation":"Family environment often provides the primary context for early identity development."},{"parent_question_id":"Q","sub_question":"How do early childhood experiences influence personality traits?","edge_annotation":"Exploring the link between childhood experiences and personality can illustrate identity formation."},{"parent_question_id":"Q","sub_question":"In what ways do education and peer interactions contribute to childhood identity?","edge_annotation":"Understanding the influence of education and peers helps clarify social and cognitive factors in identity development."},{"parent_question_id":"Q","sub_question":"How does culture influence identity

In [8]:
# count= 0
# with open("experiment_results.jsonl", 'r', encoding='utf-8') as f:
#     for line in f:
#         entry = json.loads(line)
#         if entry.get("model") == "llama 3.3 70B" and entry.get("answer") is None:
#             print("system:" + entry.get("system") + ", question_id: " + entry.get("question_id"))
#             print(entry.get("error"))
#             print()
#             count += 1
# print("count: " + str(count))

import json
from collections import Counter

# 讀取 JSON 檔案 final_llama_CQ_Solver_summary
with open('CQ_Solver_summary_2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    print(data[0]["question"])

# 提取所有 "question" 值
questions = [item['question'] for item in data]

# 計算 "question" 出現次數
question_counts = Counter(questions)

# 計算 unique "question" 的數量
unique_questions_count = len(question_counts)

# 找出重複的 "question"
duplicate_questions = [question for question, count in question_counts.items() if count > 1]

# 顯示結果
print(f"Unique 'question' count: {unique_questions_count}")
print(f"Duplicate 'questions': {duplicate_questions}")

{
  "nodes": [
    {
      "node_id": "Q",
      "question": "how did depictions of indians in popular culture help to sway popular opinion?",
      "annotation": ""
    }
  ],
  "edges": []
}
Unique 'question' count: 100
Duplicate 'questions': []


In [1]:
import json

input_file = 'llama_makeup_evaluation_results.jsonl'
output_file = 'llama_makeup_evaluation_results.jsonl'

cleaned_data = []

with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            entry = json.loads(line)
            custom_id = entry.get("custom_id")
            content = (
                entry.get("response", {})
                     .get("body", {})
                     .get("choices", [{}])[0]
                     .get("message", {})
                     .get("content", "")
            )
            cleaned_data.append({
                "custom_id": custom_id,
                "content": content
            })
        except Exception as e:
            print(f"❌ 處理失敗: {e}")
            continue

# ✅ 可選：寫入清理後的 .jsonl 檔案
with open(output_file, 'w', encoding='utf-8') as f:
    for item in cleaned_data:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ 完成，共處理 {len(cleaned_data)} 筆資料。結果已儲存至 {output_file}")


✅ 完成，共處理 491 筆資料。結果已儲存至 llama_makeup_evaluation_results.jsonl


In [2]:
import json

input_file = 'llama_makeup_evaluation_results.jsonl'
output_file = 'llama_makeup_evaluation_results.jsonl'

def parse_custom_id(custom_id: str):
    # e.g. "question_01-react" or "question_02-cq-solver"
    parts = custom_id.split("-")
    question_id = "-".join(parts[:1])  # handles formats like "question_01"
    system_raw = "-".join(parts[1:2]) if len(parts) > 2 else parts[-1]

    system_map = {
        "react": "ReAct",
        "cq-solver": "CQ_Solver"
    }

    system_name = system_map.get(system_raw.lower(), system_raw)

    return question_id, system_name

formatted = []

with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            entry = json.loads(line)
            custom_id = entry.get("custom_id", "")
            content = entry.get("content", "")

            question_id, system_name = parse_custom_id(custom_id)

            formatted.append({
                "system": system_name,
                "model": "llama 3.3 70B",# llama 3.3 70B, gpt-4o
                "question_id": question_id,
                "content": content
            })
        except Exception as e:
            print(f"❌ 錯誤處理 custom_id={entry.get('custom_id')}: {e}")
            continue

# 寫入新格式的檔案
with open(output_file, 'w', encoding='utf-8') as f:
    for item in formatted:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ 格式轉換完成，結果已寫入 {output_file}，共 {len(formatted)} 筆資料。")


✅ 格式轉換完成，結果已寫入 llama_makeup_evaluation_results.jsonl，共 491 筆資料。


In [3]:
import json
import ast
import re

input_file = 'llama_makeup_evaluation_results.jsonl'
output_file = 'llama_makeup_evaluation_results.jsonl'

# 評分欄位標準
expected_fields = {
    'Factuality', 'User Satisfaction', 'Clarity', 'Logical Coherence', 'Completeness', 'Final Score'
}

valid_scores = set(range(1, 11))

valid_entries = []
invalid_entries = []

with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        content = entry.get("content", "")

        # 1️⃣ 嘗試從文字中找出可能的字典型評分
        score_dict = None
        try:
            # 使用正則找出像 dict 的文字片段
            matches = re.findall(r"\{[^{}]+\}", content)

            for match in matches:
                try:
                    # 嘗試用 ast.literal_eval 安全解析
                    possible_dict = ast.literal_eval(match)

                    # 2️⃣ 檢查是否是 dict 且為 6 個欄位
                    if isinstance(possible_dict, dict) and len(possible_dict) == 6:
                        keys = set(possible_dict.keys())
                        values = set(possible_dict.values())

                        # 3️⃣ 檢查欄位名稱是否完全符合
                        if keys == expected_fields:

                            # 4️⃣ 檢查所有分數是否在 1~10
                            if all(isinstance(v, int) and v in valid_scores for v in possible_dict.values()):
                                score_dict = possible_dict
                                break
                except Exception:
                    continue

        except Exception as e:
            print(f"⚠️ 無法解析 content：{e}")

        # 5️⃣ 加入結果或記錄錯誤
        if score_dict:
            entry["score"] = score_dict
            valid_entries.append(entry)
        else:
            invalid_entries.append(entry)

# 寫入處理後的檔案（僅含成功解析的）
with open(output_file, 'w', encoding='utf-8') as f:
    for entry in valid_entries:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')

# 顯示報告
print(f"✅ 共處理 {len(valid_entries) + len(invalid_entries)} 筆")
print(f"✅ 符合格式的資料：{len(valid_entries)} 筆")
print(f"❌ 不符合格式的資料：{len(invalid_entries)} 筆")

# 可選：列出前幾筆錯誤來檢查原因
print("\n📌 前幾筆違反格式的 custom_id：")
for item in invalid_entries:
    print("-", item.get("system"))
    print("-", item.get("question_id"))


✅ 共處理 491 筆
✅ 符合格式的資料：491 筆
❌ 不符合格式的資料：0 筆

📌 前幾筆違反格式的 custom_id：


In [ ]:
import json

source_file = 'experiment_results.jsonl'
override_file = 'experiment_results_2.jsonl'
output_file = 'merged_experiment_results.jsonl'

# === 1. 讀取 override 資料，建立以 (system, model, question_id) 為 key 的對照表 ===
override_map = {}

with open(override_file, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        key = (entry.get("system"), entry.get("model"), entry.get("question_id"))
        override_map[key] = {
            "question": entry.get("question"),
            "answer": entry.get("answer"),
            "error": entry.get("error")
        }

# === 2. 開啟原始資料，根據 key 進行欄位覆蓋 ===
merged = []

with open(source_file, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        key = (entry.get("system"), entry.get("model"), entry.get("question_id"))

        if key in override_map:
            overrides = override_map[key]
            entry["answer"] = overrides["answer"]
            entry["error"] = overrides["error"]

        merged.append(entry)

# === 3. 寫入合併後的新檔案 ===
with open(output_file, 'w', encoding='utf-8') as f:
    for entry in merged:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ 合併完成，已寫入 {output_file}，共覆蓋 {len(override_map)} 筆資料。")


In [30]:
import json

# === 檔案路徑 ===
file_first = 'llama_evaluation_results.jsonl'
file_makeup = 'llama_makeup_evaluation_results.jsonl'
output_file = 'merge_llama_evaluation_results.jsonl'

# === 排除名單：這些 second 版本資料找不到 summary，要用 first ===
invalid_second_ids = {
    "question_07", "question_73", "question_153", "question_168",
    "question_243", "question_251", "question_260", "question_265",
    "question_278", "question_298", "question_349", "question_365",
    "question_402", "question_407", "question_474", "question_498"
}

# === 過濾條件 ===
TARGET_SYSTEM = "CQ_Solver"
TARGET_MODEL = "llama 3.3 70B"

# === 收集所有 entry
all_entries = []
question_counter = {}

def process_file(file_path, is_first_file):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)

            if entry.get("system") != TARGET_SYSTEM or entry.get("model") != TARGET_MODEL:
                continue

            qid = entry.get("question_id")
            question_counter[qid] = question_counter.get(qid, 0) + 1

            # 判定出現順序
            if question_counter[qid] == 1:
                showup = "first"
            elif question_counter[qid] == 2:
                showup = "second"
            elif question_counter[qid] == 3:
                showup = "third"
            else:
                showup = f"repeat{question_counter[qid]}"

            # 若是無效的 second，跳過
            if qid in invalid_second_ids and showup == "second":
                continue

            entry["showup"] = showup
            all_entries.append(entry)

# === 處理兩份來源檔案 ===
process_file(file_first, is_first_file=True)
process_file(file_makeup, is_first_file=False)

# === 選出最佳版本 ===
best_by_question = {}

def score_key(entry):
    s = entry["score"]
    total = sum(v for k, v in s.items() if k != "Final Score")
    return (s["Final Score"], total, s["Factuality"])

for entry in all_entries:
    qid = entry["question_id"]
    current_score = score_key(entry)

    if qid not in best_by_question:
        best_by_question[qid] = entry
    else:
        existing = best_by_question[qid]
        if current_score > score_key(existing):
            best_by_question[qid] = entry
        # 若同分，保留最早出現

# === 寫入合併後的最佳版本 ===
with open(output_file, 'w', encoding='utf-8') as f:
    for qid, entry in best_by_question.items():
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')

print(f"✅ 修正與合併完成，寫入 {output_file}，共保留 {len(best_by_question)} 筆資料。")


✅ 修正與合併完成，寫入 merge_llama_evaluation_results.jsonl，共保留 500 筆資料。


In [ ]:
import json

# === 已完成的結果檔案 ===
final_experiment_file = 'final_experiment_results.jsonl'
output_summary_file = 'final_llama_CQ_Solver_summary.json'

# === 原始 summary 資料來源 ===
summary_files = {
    "first": "llama_summary.json",
    "second": "llama_CQ_Solver_summary_makeup.json",
    "third": "llama_CQ_Solver_summary_makeup2.json"
}

# === 建立 summary 索引：使用 (question_id, showup) 作為唯一 key ===
summary_data_by_qid = {}

def extract_question_text(q_graph_str):
    try:
        graph = json.loads(q_graph_str)
        return graph["nodes"][0]["question"]
    except Exception:
        return None

for label, file_path in summary_files.items():
    with open(file_path, 'r', encoding='utf-8') as f:
        summaries = json.load(f)
        for s in summaries:
            question_text = extract_question_text(s["question"])
            if not question_text:
                continue
            summary_data_by_qid[(question_text, label)] = s  # 暫時以 question_text 為中介

# === 重新建立 (question_id, label) → summary_entry 的 mapping ===
# 先讀 final_experiment_results.jsonl 中 CQ_Solver / llama 的內容來建立最終匹配
final_summaries = []

with open(final_experiment_file, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        system = entry.get("system")
        model = entry.get("model")
        question_id = entry.get("question_id")
        question_text = entry.get("question")

        if system != "CQ_Solver" or model != "llama 3.3 70B":
            continue

        # 判斷它是來自哪個版本（依據原來標記）
        showup = entry.get("__source__", "first")

        # 使用 (question_text, source_label) 找 summary
        summary_entry = summary_data_by_qid.get((question_text, showup))

        if summary_entry:
            final_summaries.append(summary_entry)
        else:
            print(f"⚠️ 無法找到對應摘要：{question_id} ({showup})")

# === 寫入修正後的 summary 檔案 ===
with open(output_summary_file, 'w', encoding='utf-8') as f:
    json.dump(final_summaries, f, ensure_ascii=False, indent=2)

# === 驗證筆數 ===
print(f"✅ 已完成 {output_summary_file}，共收錄 {len(final_summaries)} 筆摘要")


question_07 (second)
question_73 (second)
question_153 (second)
question_168 (second)
question_243 (second)
question_251 (second)
question_260 (second)
question_265 (second)
question_278 (second)
question_298 (second)
question_349 (second)
question_365 (second)
question_402 (second)
question_407 (second)
question_474 (second)
question_498 (second)
✅ 已完成 final_llama_CQ_Solver_summary.json，共收錄 484 筆摘要


In [31]:
import json

# === 檔案路徑 ===
eval_file = 'merge_llama_evaluation_results.jsonl'
output_experiment_file = 'final_experiment_results.jsonl'
output_summary_file = 'final_llama_CQ_Solver_summary.json'

# === 原始來源 ===
experiment_files = {
    "first": "experiment_results.jsonl",
    "second": "llama_experiment_results_makeup_exam.jsonl",
    "third": "llama_experiment_results_makeup_exam2.jsonl"
}

summary_files = {
    "first": "llama_summary.json",
    "second": "llama_CQ_Solver_summary_makeup.json",
    "third": "llama_CQ_Solver_summary_makeup2.json"
}

# === 讀取最佳評分資料 ===
best_versions = {}
with open(eval_file, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        key = (entry["system"], entry["model"], entry["question_id"])
        best_versions[key] = {
            "showup": entry["showup"]
        }

# === 建立回答資料索引: (system, model, question_id) -> entry ===
experiment_data = {}

for label, file_path in experiment_files.items():
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            key = (entry["system"], entry["model"], entry["question_id"])
            entry["__source__"] = label
            experiment_data[key, label] = entry

# === 建立摘要資料索引: question_text → entry ===
summary_data_by_qid = {}

def extract_question_text(graph_json_str):
    try:
        graph = json.loads(graph_json_str)
        return graph["nodes"][0]["question"]
    except Exception:
        return None

for label, file_path in summary_files.items():
    with open(file_path, 'r', encoding='utf-8') as f:
        summaries = json.load(f)
        for s in summaries:
            q_text = extract_question_text(s["question"])
            if not q_text:
                continue
            # 反查對應的 question_id：從 experiment_data 中找 match
            for (key, lbl), exp_entry in experiment_data.items():
                if lbl != label:
                    continue
                if exp_entry["question"] == q_text:
                    qid = key[2]  # (system, model, question_id)
                    summary_data_by_qid[(qid, label)] = s
                    break

# === 合併生成 final_experiment_results.jsonl ===
with open(output_experiment_file, 'w', encoding='utf-8') as f_out:
    for (key, label), entry in experiment_data.items():
        if key not in best_versions or best_versions[key]["showup"] == label:
            json.dump(entry, f_out, ensure_ascii=False)
            f_out.write('\n')

print(f"✅ 已完成 {output_experiment_file}")

# === 合併生成 final_llama_CQ_Solver_summary.json ===
final_summaries = []

for key, meta in best_versions.items():
    system, model, question_id = key
    if system != "CQ_Solver" or model != "llama 3.3 70B":
        continue

    # 找出對應的 question text（從任一來源的回答 entry）
    # 找任意 label 都行，取出 question text
    entry = experiment_data.get((key, meta["showup"]))
    if not entry:
        continue

    question_text = entry["question"]

    summary_entry = summary_data_by_qid.get((question_id, meta["showup"]))
    if summary_entry:
        final_summaries.append(summary_entry)

# === 寫入摘要
with open(output_summary_file, 'w', encoding='utf-8') as f_out:
    json.dump(final_summaries, f_out, ensure_ascii=False, indent=2)

print(f"✅ 已完成 {output_summary_file}")


✅ 已完成 final_experiment_results.jsonl
✅ 已完成 final_llama_CQ_Solver_summary.json


In [14]:
import json
from collections import defaultdict

input_file = "refined_evaluation_results.jsonl" #'./result/final_evaluation_results.jsonl'

expected_fields = [
    'Factuality', 'User Satisfaction', 'Clarity',
    'Logical Coherence', 'Completeness', 'Final Score'
]

# 初始化統計資料: (system, model) → 各評分欄位的數值 list
systems_scores = defaultdict(lambda: defaultdict(list))

# 讀取並分類累加
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        system = entry.get("system")
        model = entry.get("model")
        score = entry.get("score")

        if system and model and score:
            key = (system, model)
            for field in expected_fields:
                value = score.get(field)
                if isinstance(value, int):
                    systems_scores[key][field].append(value)

# 計算平均分數
print("\n📊 各系統 + 模型 的平均成績：\n")
for (system, model), scores in systems_scores.items():
    print(f"🔹 系統：{system} | 模型：{model}")
    for field in expected_fields:
        values = scores[field]
        if values:
            avg = sum(values) / len(values)
            print(f"  {field}: {avg:.2f}")
        else:
            print(f"  {field}: 無資料")
    print()



📊 各系統 + 模型 的平均成績：

🔹 系統：CQ_Solver | 模型：llama 3.3 70B
  Factuality: 6.99
  User Satisfaction: 6.05
  Clarity: 7.58
  Logical Coherence: 7.51
  Completeness: 5.62
  Final Score: 6.42



In [13]:
import json
from collections import defaultdict

input_file = './result/final_evaluation_results.jsonl'
target_system = "CQ_Solver"

expected_fields = [
    'Factuality', 'User Satisfaction', 'Clarity',
    'Logical Coherence', 'Completeness', 'Final Score'
]

# 建立資料結構：每個欄位，每個分數，對應的 question_id 清單
score_distribution = {
    field: defaultdict(list)
    for field in expected_fields
}

# 分析 CQ_Solver 的資料
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        if entry.get("system") != target_system or entry.get("model") != "llama 3.3 70B":
            continue

        question_id = entry.get("question_id", "unknown")
        score = entry.get("score", {})

        for field in expected_fields:
            value = score.get(field)
            if isinstance(value, int) and 1 <= value <= 10:
                score_distribution[field][value].append(question_id)

# 顯示結果
print(f"\n📊 分析系統：{target_system} 的分數組成\n")

for field in expected_fields:
    print(f"🔹 評分項目：{field}")
    for score_value in range(10, 0, -1):
        questions = score_distribution[field].get(score_value, [])
        print(f"  分數 {score_value}: {len(questions)} 筆")
        if questions:
            print(f"    ➤ 問題 IDs: {', '.join(questions)}")
    print()



📊 分析系統：CQ_Solver 的分數組成

🔹 評分項目：Factuality
  分數 10: 1 筆
    ➤ 問題 IDs: question_323
  分數 9: 16 筆
    ➤ 問題 IDs: question_54, question_110, question_141, question_142, question_157, question_239, question_241, question_316, question_329, question_366, question_371, question_392, question_409, question_473, question_336, question_425
  分數 8: 136 筆
    ➤ 問題 IDs: question_03, question_05, question_09, question_14, question_16, question_17, question_19, question_23, question_28, question_30, question_33, question_34, question_37, question_39, question_40, question_44, question_45, question_50, question_51, question_60, question_69, question_70, question_74, question_75, question_76, question_79, question_81, question_91, question_99, question_100, question_102, question_111, question_112, question_114, question_116, question_117, question_118, question_119, question_120, question_123, question_128, question_129, question_131, question_136, question_140, question_146, question_151, question_15

In [4]:
import json

input_file = 'llama_evaluation_results.jsonl'  # 'scored_results.jsonl'
target_system = "CQ_Solver"

# 儲存目標問題的 question_id
score_5_ids = []
score_6_ids = []
score_7_all = []

# with open(input_file, 'r', encoding='utf-8') as f:
#     for line in f:
#         entry = json.loads(line)
#         if entry.get("system") != target_system:
#             continue

#         score = entry.get("score", {})
#         final_score = score.get("Final Score")
#         question_id = entry.get("question_id", "unknown")

#         if final_score == 5:
#             score_5_ids.append(question_id)
#         elif final_score == 6:
#             score_6_ids.append(question_id)
#         elif final_score == 7:
#             entry["score_sum"] = sum(score[field] for field in score if field != "Final Score")
#             score_7_all.append(entry)

# # 對 Final Score = 7 的進行排序並取後 50%
# score_7_all_sorted = sorted(score_7_all, key=lambda x: x["score_sum"])
# mid = len(score_7_all_sorted) // 2
# score_7_weak_ids = [e["question_id"] for e in score_7_all_sorted[mid:]]

# # === 印出結果 ===

# print(score_5_ids + score_6_ids + score_7_weak_ids)

low_score_ids = []
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        entry = json.loads(line)
        if entry.get("system") != target_system:
            continue

        score = entry.get("score", {})
        final_score = score.get("Final Score")
        question_id = entry.get("question_id", "unknown")

        if final_score <= 6:
            low_score_ids.append(question_id)

print(low_score_ids)

low_score_q_1 = ['question_02', 'question_03', 'question_04', 'question_07', 'question_10', 'question_13', 'question_18', 'question_19', 'question_21', 'question_22', 'question_24', 'question_25', 'question_28', 'question_31', 'question_32', 'question_36', 'question_38', 'question_39', 'question_40', 'question_43', 'question_48', 'question_50', 'question_52', 'question_54', 'question_56', 'question_61', 'question_62', 'question_65', 'question_70', 'question_71', 'question_72', 'question_73', 'question_76', 'question_77', 'question_83', 'question_87', 'question_89', 'question_91', 'question_92', 'question_93', 'question_96', 'question_100', 'question_102', 'question_103', 'question_104', 'question_106', 'question_107', 'question_108', 'question_109', 'question_110', 'question_111', 'question_113', 'question_116', 'question_118', 'question_119', 'question_121', 'question_122', 'question_124', 'question_126', 'question_127', 'question_128', 'question_131', 'question_133', 'question_134', 'question_135', 'question_136', 'question_137', 'question_138', 'question_139', 'question_141', 'question_142', 'question_143', 'question_144', 'question_145', 'question_147', 'question_152', 'question_153', 'question_154', 'question_155', 'question_159', 'question_160', 'question_165', 'question_167', 'question_168', 'question_172', 'question_174', 'question_177', 'question_182', 'question_183', 'question_184', 'question_189', 'question_192', 'question_193', 'question_194', 'question_199', 'question_200', 'question_203', 'question_209', 'question_210', 'question_211', 'question_214', 'question_215', 'question_217', 'question_218', 'question_221', 'question_222', 'question_224', 'question_227', 'question_230', 'question_231', 'question_232', 'question_234', 'question_235', 'question_237', 'question_238', 'question_241', 'question_243', 'question_245', 'question_246', 'question_247', 'question_249', 'question_251', 'question_252', 'question_254', 'question_255', 'question_257', 'question_259', 'question_260', 'question_262', 'question_264', 'question_265', 'question_266', 'question_269', 'question_270', 'question_271', 'question_272', 'question_275', 'question_276', 'question_278', 'question_279', 'question_280', 'question_281',
                 'question_282', 'question_283', 'question_286', 'question_287', 'question_293', 'question_294', 'question_298', 'question_299', 'question_300', 'question_301', 'question_302', 'question_305', 'question_306', 'question_307', 'question_308', 'question_309', 'question_310', 'question_311', 'question_312', 'question_313', 'question_315', 'question_316', 'question_318', 'question_319', 'question_320', 'question_321', 'question_322', 'question_324', 'question_325', 'question_326', 'question_328', 'question_331', 'question_332', 'question_335', 'question_338', 'question_343', 'question_344', 'question_345', 'question_349', 'question_351', 'question_352', 'question_353', 'question_354', 'question_356', 'question_357', 'question_358', 'question_361', 'question_362', 'question_363', 'question_365', 'question_370', 'question_371', 'question_373', 'question_375', 'question_376', 'question_380', 'question_383', 'question_385', 'question_387', 'question_388', 'question_391', 'question_395', 'question_396', 'question_397', 'question_399', 'question_402', 'question_403', 'question_405', 'question_406', 'question_407', 'question_408', 'question_409', 'question_411', 'question_414', 'question_415', 'question_418', 'question_419', 'question_422', 'question_428', 'question_429', 'question_430', 'question_431', 'question_432', 'question_433', 'question_438', 'question_441', 'question_442', 'question_444', 'question_445', 'question_446', 'question_447', 'question_448', 'question_449', 'question_451', 'question_454', 'question_456', 'question_459', 'question_460', 'question_461', 'question_463', 'question_465', 'question_468', 'question_472', 'question_474', 'question_476', 'question_480', 'question_481', 'question_482', 'question_485', 'question_487', 'question_489', 'question_490', 'question_492', 'question_493', 'question_494', 'question_496', 'question_497', 'question_498', 'question_41', 'question_53', 'question_55', 'question_90', 'question_97', 'question_188', 'question_196', 'question_206', 'question_212', 'question_216', 'question_334', 'question_336', 'question_382', 'question_390', 'question_393', 'question_410', 'question_417', 'question_466', 'question_475', 'question_163', 'question_66', 'question_125']

['question_02', 'question_03', 'question_04', 'question_07', 'question_10', 'question_13', 'question_18', 'question_19', 'question_21', 'question_22', 'question_24', 'question_25', 'question_28', 'question_31', 'question_32', 'question_36', 'question_38', 'question_39', 'question_40', 'question_43', 'question_48', 'question_50', 'question_52', 'question_54', 'question_56', 'question_61', 'question_62', 'question_65', 'question_70', 'question_71', 'question_72', 'question_73', 'question_76', 'question_77', 'question_83', 'question_87', 'question_89', 'question_91', 'question_92', 'question_93', 'question_96', 'question_100', 'question_102', 'question_103', 'question_104', 'question_106', 'question_107', 'question_108', 'question_109', 'question_110', 'question_111', 'question_113', 'question_116', 'question_118', 'question_119', 'question_121', 'question_122', 'question_124', 'question_126', 'question_127', 'question_128', 'question_131', 'question_133', 'question_134', 'question_135', 

In [13]:
low_score_q_1 = ['question_02', 'question_03', 'question_04', 'question_07', 'question_10', 'question_13', 'question_18', 'question_19', 'question_21', 'question_22', 'question_24', 'question_25', 'question_28', 'question_31', 'question_32', 'question_36', 'question_38', 'question_39', 'question_40', 'question_43', 'question_48', 'question_50', 'question_52', 'question_54', 'question_56', 'question_61', 'question_62', 'question_65', 'question_70', 'question_71', 'question_72', 'question_73', 'question_76', 'question_77', 'question_83', 'question_87', 'question_89', 'question_91', 'question_92', 'question_93', 'question_96', 'question_100', 'question_102', 'question_103', 'question_104', 'question_106', 'question_107', 'question_108', 'question_109', 'question_110', 'question_111', 'question_113', 'question_116', 'question_118', 'question_119', 'question_121', 'question_122', 'question_124', 'question_126', 'question_127', 'question_128', 'question_131', 'question_133', 'question_134', 'question_135', 'question_136', 'question_137', 'question_138', 'question_139', 'question_141', 'question_142', 'question_143', 'question_144', 'question_145', 'question_147', 'question_152', 'question_153', 'question_154', 'question_155', 'question_159', 'question_160', 'question_165', 'question_167', 'question_168', 'question_172', 'question_174', 'question_177', 'question_182', 'question_183', 'question_184', 'question_189', 'question_192', 'question_193', 'question_194', 'question_199', 'question_200', 'question_203', 'question_209', 'question_210', 'question_211', 'question_214', 'question_215', 'question_217', 'question_218', 'question_221', 'question_222', 'question_224', 'question_227', 'question_230', 'question_231', 'question_232', 'question_234', 'question_235', 'question_237', 'question_238', 'question_241', 'question_243', 'question_245', 'question_246', 'question_247', 'question_249', 'question_251', 'question_252', 'question_254', 'question_255', 'question_257', 'question_259', 'question_260', 'question_262', 'question_264', 'question_265', 'question_266', 'question_269', 'question_270', 'question_271', 'question_272', 'question_275', 'question_276', 'question_278', 'question_279', 'question_280', 'question_281',
                 'question_282', 'question_283', 'question_286', 'question_287', 'question_293', 'question_294', 'question_298', 'question_299', 'question_300', 'question_301', 'question_302', 'question_305', 'question_306', 'question_307', 'question_308', 'question_309', 'question_310', 'question_311', 'question_312', 'question_313', 'question_315', 'question_316', 'question_318', 'question_319', 'question_320', 'question_321', 'question_322', 'question_324', 'question_325', 'question_326', 'question_328', 'question_331', 'question_332', 'question_335', 'question_338', 'question_343', 'question_344', 'question_345', 'question_349', 'question_351', 'question_352', 'question_353', 'question_354', 'question_356', 'question_357', 'question_358', 'question_361', 'question_362', 'question_363', 'question_365', 'question_370', 'question_371', 'question_373', 'question_375', 'question_376', 'question_380', 'question_383', 'question_385', 'question_387', 'question_388', 'question_391', 'question_395', 'question_396', 'question_397', 'question_399', 'question_402', 'question_403', 'question_405', 'question_406', 'question_407', 'question_408', 'question_409', 'question_411', 'question_414', 'question_415', 'question_418', 'question_419', 'question_422', 'question_428', 'question_429', 'question_430', 'question_431', 'question_432', 'question_433', 'question_438', 'question_441', 'question_442', 'question_444', 'question_445', 'question_446', 'question_447', 'question_448', 'question_449', 'question_451', 'question_454', 'question_456', 'question_459', 'question_460', 'question_461', 'question_463', 'question_465', 'question_468', 'question_472', 'question_474', 'question_476', 'question_480', 'question_481', 'question_482', 'question_485', 'question_487', 'question_489', 'question_490', 'question_492', 'question_493', 'question_494', 'question_496', 'question_497', 'question_498', 'question_41', 'question_53', 'question_55', 'question_90', 'question_97', 'question_188', 'question_196', 'question_206', 'question_212', 'question_216', 'question_334', 'question_336', 'question_382', 'question_390', 'question_393', 'question_410', 'question_417', 'question_466', 'question_475', 'question_163', 'question_66', 'question_125']
len(low_score_q_1)

282

In [10]:
import os
import json

# === 檔案來源 ===
scored_results_file = 'merged_scored_results.jsonl'
llama_makeup_file = 'llama_evaluation_results.jsonl'
llama_merge_file = 'merge_llama_evaluation_results.jsonl'
output_file = './result/final_evaluation_results.jsonl'

# ✅ 確保資料夾存在
os.makedirs('./result', exist_ok=True)

# === 開啟輸出檔案開始寫入 ===
with open(output_file, 'w', encoding='utf-8') as fout:

    # 1️⃣ 複製 merged_scored_results.jsonl 的所有資料
    with open(scored_results_file, 'r', encoding='utf-8') as f1:
        for line in f1:
            fout.write(line)

    # 2️⃣ 複製 llama_makeup_evaluation_results.jsonl 的非 CQ_Solver / llama 3.3 70B 資料
    with open(llama_makeup_file, 'r', encoding='utf-8') as f2:
        for line in f2:
            entry = json.loads(line)
            if entry.get("system") == "CQ_Solver" and entry.get("model") == "llama 3.3 70B":
                continue
            fout.write(json.dumps(entry, ensure_ascii=False) + '\n')

    # 3️⃣ 複製 merge_llama_evaluation_results.jsonl 的所有資料
    with open(llama_merge_file, 'r', encoding='utf-8') as f3:
        for line in f3:
            fout.write(line)

print(f"✅ 已完成合併，輸出至 {output_file}")


✅ 已完成合併，輸出至 ./result/final_evaluation_results.jsonl
